In [52]:
#Given a year 2009
# we have N cells 
# Each cell contains information [{sst,wind_dir, cholorphyll, index, month, month_index}] (array is sorted by month)


In [67]:
import math
import json
from pprint import pprint
import pandas as pd
from datetime import datetime

In [54]:
def drange(start, stop, step):
    r = start
    while r < stop:
        yield r
        r += step

In [92]:
def readSSTData(year):
    data = []
    filename = "../data/sst" + str(year) + '.json'
    with open(filename) as f:
        data = json.load(f)
    return data

In [93]:
def readWindData():
    data = []
    filename = "../data/wind.json"
    with open(filename) as f:
        data = json.load(f)
    return data
    

In [94]:
def initMatrix(yearRange):
    matrix = []
    smallestLat = 35.775
    largestLat = 39.15
    smallestLon = 235.5625
    largestLon = 238.9375
    latRange = largestLat - smallestLat
    lonRange = largestLon - smallestLon

    latStep = latRange/10
    lonStep = lonRange/10

    latInterval = drange(smallestLat, largestLat, latStep)
    latInc = [float("{0:.4f}".format(x)) for x in latInterval]
    latInc = latInc[:-1]

    lonInterval = drange(smallestLon, largestLon, lonStep)
    lonInc = [float("{0:.4f}".format(x)) for x in lonInterval]
    
    for lat_ in latInc:
        for lon_ in lonInc:
            latRnge =[lat_,lat_+1]
            lonRnge =[lon_,lon_+1]
            cell = {'latRange': latRnge, 'lonRange': lonRnge}
            for year in yearRange:
                cell[year] = {'sst':-9999,'windDegree':-9999}
            matrix.append(cell)
        
    return matrix
    

In [ ]:
def retrieveNOAAData(dateStart, dateEnd){
        fetch(....[date])
    https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplMURSST41mday.json?sst[
        (2018-12-16T00:00:00Z):1:(2018-12-16T00:00:00Z)]
    [(-89.99):1:(89.99)][(-179.99):1:(180.0)],nobs[(2018-12-16T00:00:00Z):1:(2018-12-16T00:00:00Z)][(-89.99):1:(89.99)][(-179.99):1:(180.0)],mask[(2018-12-16T00:00:00Z):1:(2018-12-16T00:00:00Z)][(-89.99):1:(89.99)][(-179.99):1:(180.0)]
}

In [99]:
#extend this to handle days for that month.
# e.g sst:(float) -> sst:[float,float,...] (sorted by day)
def processFile(years,matrix):
    for year in years:
        sstData = readSSTData(year)
        rows = sstData["table"]["rows"]
        for idx, cell in enumerate(matrix):
            cell = matrix[idx]
            totalIntTemp = 0
            count = 0
            avgIntTemp = 0
            for row in rows:
                lat = row[2]
                lon = row[3]
                sst = row[4]
                if((cell['latRange'][0] <= lat <= cell['latRange'][1] ) and (cell['lonRange'][0] <= lon <= cell['lonRange'][1])):
                    if(sst != None and sst != 0):
                        count = count + 1
                        totalIntTemp = totalIntTemp + sst
            if count != 0:
                avgIntTemp = totalIntTemp/count
                cell[year]['sst'] = float("{0:.4f}".format(avgIntTemp))
        
    return matrix

In [100]:
def processWindData(matrix):
    windData = readWindData()
    windRows = windData['table']['rows']
    for idx, cell in enumerate(matrix):
        cell = matrix[idx]
        for wRow in windRows:
            datetime_object = datetime.strptime(wRow[0], '%Y-%m-%dT%H:%M:%SZ')
            year = datetime_object.year
            lat = wRow[2]
            lon = wRow[3]
            xWind = wRow[4]
            yWind = wRow[5]
            if((cell['latRange'][0] <= lat <= cell['latRange'][1] ) and (cell['lonRange'][0] <= lon <= cell['lonRange'][1])):
                if(xWind != None and yWind != None):
                    cell[year]['windDegree'] = wind_degree = math.degrees(math.atan(yWind/xWind))
    return matrix

In [103]:
#run this function for all 
years = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
sstYears = [2009,2010,2011,2015,2016,2017,2018]
matrix = initMatrix(years)
matrix = processFile(sstYears,matrix)
matrix = processWindData(matrix)

In [105]:
with open('data.json', 'w') as outfile:
    json.dump(matrix, outfile)